In [1]:
! pip install finance-datareader

In [2]:
import FinanceDataReader as fdr
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import itertools
import random

from sklearn.preprocessing import MinMaxScaler, StandardScaler

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense,  GRU, Concatenate, Dropout, LSTM, SimpleRNN, TimeDistributed
from tensorflow.keras import Sequential, Input, Model, layers, optimizers
from tensorflow.keras.optimizers import SGD, Adam

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/stock_2/'
list_name = 'stock_list.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [5]:
def LSTM_model(X_train, y_train, X_val, y_val):

  model = Sequential()
  model.add(LSTM(units = 16, input_shape = (X_train.shape[1], 1), activation = 'tanh', return_sequences = True))
  model.add(Dropout(rate = 0.2))
  model.add(LSTM(units = 16, activation = 'tanh', return_sequences = True))
  model.add(Dropout(rate = 0.2))
  model.add(LSTM(units = 16, activation = 'tanh', return_sequences = True))
  model.add(Dropout(rate = 0.2))
  model.add(TimeDistributed(Dense(units = 1)))

  model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'mean_absolute_error', metrics=['mae']) 

  es = EarlyStopping(monitor ='val_loss', mode ='min', verbose = 1, patience = 5)
  model_save_folder_path = '/content/drive/MyDrive/stock_2/lstm_public/lstm'
  model_path = model_save_folder_path + f'{code}.h5'
  mc = ModelCheckpoint(model_path, monitor ='val_loss', mode ='min', verbose = 100, save_best_only = True)

  model.fit(X_train, y_train, epochs = 500, batch_size = 2, verbose = 1, shuffle = False, callbacks = [es, mc], validation_data=(X_val, y_val))

  return model

In [6]:
submission_name = 'sample_submission.csv'
submission = pd.read_csv(os.path.join(path,submission_name))

In [7]:
start_date = '20210104'
end_date = '20211105'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

In [8]:
for code in tqdm(stock_list['종목코드'].values):
  
  # 해당 종목의 데이터 가져온 후 주 단위 데이터로 나타내기
  start_date = '20210104'
  end_date = '20211105'
  stock_price = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
  stock_price = pd.merge(business_days, stock_price, how = 'outer')
  stock_price['weekday'] = stock_price.Date.apply(lambda x : x.weekday())
  stock_price['weeknum'] = stock_price.Date.apply(lambda x : x.strftime('%V'))
  stock_price.Close = stock_price.Close.ffill()
  stock_price = pd.pivot_table(data = stock_price, values = 'Close', columns = 'weekday', index = 'weeknum')

  X_train = stock_price.iloc[0:-12].to_numpy()
  X_val = stock_price.iloc[-12:-2].to_numpy()
  X_test = stock_price.iloc[-2].to_numpy()

  y_train = stock_price.iloc[1:-11].to_numpy()
  y_val = stock_price.iloc[-11:-1].to_numpy()
  y_test = stock_price.iloc[-1].to_numpy()

  # 스케일링 시행
  scaler = MinMaxScaler()

  X_train_scaled = scaler.fit_transform(X_train.reshape(-1, 1)).reshape(32, 5)
  X_val_scaled = scaler.fit_transform(X_val.reshape(-1, 1)).reshape(10, 5)
  X_test_scaled = scaler.fit_transform(X_test.reshape(-1, 1)).reshape(1, 5)

  y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(32, 5)
  y_val_scaled = scaler.fit_transform(y_val.reshape(-1, 1)).reshape(10, 5)
  y_test_scaled = scaler.fit_transform(y_test.reshape(-1, 1)).reshape(1, 5)

  X_train_scaled = np.reshape(X_train_scaled, (X_train_scaled.shape[0], X_train_scaled.shape[1]))
  X_val_scaled = np.reshape(X_val_scaled, (X_val_scaled.shape[0], X_val_scaled.shape[1]))
  X_test_scaled = np.reshape(X_test_scaled, (X_test_scaled.shape[0], X_test_scaled.shape[1]))
  
  # 모델 학습
  model = LSTM_model(X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled)
  model = keras.models.load_model(f"/content/drive/MyDrive/stock_2/lstm_public/lstm{code}.h5")

  # 예측
  y_pred = model.predict(X_test_scaled)
  y_pred = y_pred.reshape(1,5)
  y_pred = scaler.inverse_transform(y_pred)

  print(y_pred)
    
  for i in range (0, 5):
    submission.loc[i, code] = y_pred[0][i]
    # submission.loc[i + 5, code] = y_pred[0][i] # 임시로 채워주기 위한 용도. 최종 예측 시에는 바꿔주어야 함!

12/16 [=====================>........] - ETA: 0s - loss: 0.1086 - mae: 0.1086
Epoch 00011: val_loss did not improve from 0.18286
16/16 [==============================] - 0s 13ms/step - loss: 0.1078 - mae: 0.1078 - val_loss: 0.1881 - val_mae: 0.1881
Epoch 12/500
12/16 [=====================>........] - ETA: 0s - loss: 0.1558 - mae: 0.1558
Epoch 00012: val_loss did not improve from 0.18286
16/16 [==============================] - 0s 13ms/step - loss: 0.1502 - mae: 0.1502 - val_loss: 0.2305 - val_mae: 0.2305
Epoch 13/500
11/16 [===================>..........] - ETA: 0s - loss: 0.1445 - mae: 0.1445
Epoch 00013: val_loss improved from 0.18286 to 0.15833, saving model to /content/drive/MyDrive/stock_2/lstm_public/lstm216080.h5
16/16 [==============================] - 0s 16ms/step - loss: 0.1376 - mae: 0.1376 - val_loss: 0.1583 - val_mae: 0.1583
Epoch 14/500
12/16 [=====================>........] - ETA: 0s - loss: 0.1256 - mae: 0.1256
Epoch 00014: val_loss did not improve from 0.15833
16/16 [

 87%|████████▋ | 321/370 [1:27:46<12:20, 15.11s/it]

[[27304.168 27187.398 27261.727 27246.281 27267.186]]
Epoch 1/500
11/16 [===================>..........] - ETA: 0s - loss: 0.1791 - mae: 0.1791
Epoch 00001: val_loss improved from inf to 0.30301, saving model to /content/drive/MyDrive/stock_2/lstm_public/lstm013120.h5
16/16 [==============================] - 6s 100ms/step - loss: 0.1993 - mae: 0.1993 - val_loss: 0.3030 - val_mae: 0.3030
Epoch 2/500
13/16 [=======================>......] - ETA: 0s - loss: 0.3578 - mae: 0.3578
Epoch 00002: val_loss did not improve from 0.30301
16/16 [==============================] - 0s 12ms/step - loss: 0.3705 - mae: 0.3705 - val_loss: 0.3157 - val_mae: 0.3157
Epoch 3/500
12/16 [=====================>........] - ETA: 0s - loss: 0.2705 - mae: 0.2705
Epoch 00003: val_loss improved from 0.30301 to 0.28161, saving model to /content/drive/MyDrive/stock_2/lstm_public/lstm013120.h5
16/16 [==============================] - 0s 17ms/step - loss: 0.2842 - mae: 0.2842 - val_loss: 0.2816 - val_mae: 0.2816
Epoch 4/50

 87%|████████▋ | 322/370 [1:28:01<11:57, 14.94s/it]

[[5264.9146 5334.905  5304.655  5296.1904 5263.875 ]]
Epoch 1/500
12/16 [=====================>........] - ETA: 0s - loss: 0.0617 - mae: 0.0617
Epoch 00001: val_loss improved from inf to 0.37404, saving model to /content/drive/MyDrive/stock_2/lstm_public/lstm095660.h5
16/16 [==============================] - 6s 100ms/step - loss: 0.1251 - mae: 0.1251 - val_loss: 0.3740 - val_mae: 0.3740
Epoch 2/500
12/16 [=====================>........] - ETA: 0s - loss: 0.0615 - mae: 0.0615
Epoch 00002: val_loss did not improve from 0.37404
16/16 [==============================] - 0s 13ms/step - loss: 0.1342 - mae: 0.1342 - val_loss: 0.4373 - val_mae: 0.4373
Epoch 3/500
12/16 [=====================>........] - ETA: 0s - loss: 0.0407 - mae: 0.0407
Epoch 00003: val_loss did not improve from 0.37404
16/16 [==============================] - 0s 14ms/step - loss: 0.1090 - mae: 0.1090 - val_loss: 0.3967 - val_mae: 0.3967
Epoch 4/500
16/16 [==============================] - ETA: 0s - loss: 0.1155 - mae: 0.115

...

[[7000.2163 7034.061  7092.814  7154.796  7171.988 ]]
Epoch 1/500
16/16 [==============================] - ETA: 0s - loss: 0.2099 - mae: 0.2099
Epoch 00001: val_loss improved from inf to 0.37181, saving model to /content/drive/MyDrive/stock_2/lstm_public/lstm110790.h5
16/16 [==============================] - 6s 101ms/step - loss: 0.2099 - mae: 0.2099 - val_loss: 0.3718 - val_mae: 0.3718
Epoch 2/500
12/16 [=====================>........] - ETA: 0s - loss: 0.3322 - mae: 0.3322
Epoch 00002: val_loss improved from 0.37181 to 0.23920, saving model to /content/drive/MyDrive/stock_2/lstm_public/lstm110790.h5
16/16 [==============================] - 0s 15ms/step - loss: 0.3481 - mae: 0.3481 - val_loss: 0.2392 - val_mae: 0.2392
Epoch 3/500
15/16 [===========================>..] - ETA: 0s - loss: 0.2268 - mae: 0.2268
Epoch 00003: val_loss did not improve from 0.23920
16/16 [==============================] - 0s 14ms/step - loss: 0.2335 - mae: 0.2335 - val_loss: 0.2467 - val_mae: 0.2467
Epoch 4/50

100%|█████████▉| 369/370 [1:40:09<00:14, 14.39s/it]

[[41825.734 42044.473 41853.254 41750.098 41712.668]]
Epoch 1/500
16/16 [==============================] - ETA: 0s - loss: 0.2850 - mae: 0.2850
Epoch 00001: val_loss improved from inf to 0.29808, saving model to /content/drive/MyDrive/stock_2/lstm_public/lstm099320.h5
16/16 [==============================] - 6s 104ms/step - loss: 0.2850 - mae: 0.2850 - val_loss: 0.2981 - val_mae: 0.2981
Epoch 2/500
12/16 [=====================>........] - ETA: 0s - loss: 0.2229 - mae: 0.2229
Epoch 00002: val_loss improved from 0.29808 to 0.24990, saving model to /content/drive/MyDrive/stock_2/lstm_public/lstm099320.h5
16/16 [==============================] - 0s 16ms/step - loss: 0.1899 - mae: 0.1899 - val_loss: 0.2499 - val_mae: 0.2499
Epoch 3/500
15/16 [===========================>..] - ETA: 0s - loss: 0.1557 - mae: 0.1557
Epoch 00003: val_loss did not improve from 0.24990
16/16 [==============================] - 0s 16ms/step - loss: 0.1496 - mae: 0.1496 - val_loss: 0.2535 - val_mae: 0.2535
Epoch 4/50

100%|██████████| 370/370 [1:40:29<00:00, 16.30s/it]

[[49044.418 48624.6   47247.355 48158.164 49265.543]]


In [9]:
start_date = '20210104'
end_date = '20211126'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

In [13]:
def LSTM_model(X_train, y_train, X_val, y_val):

  model = Sequential()
  model.add(LSTM(units = 16, input_shape = (X_train.shape[1], 1), activation = 'tanh', return_sequences = True))
  model.add(Dropout(rate = 0.2))
  model.add(LSTM(units = 16, activation = 'tanh', return_sequences = True))
  model.add(Dropout(rate = 0.2))
  model.add(LSTM(units = 16, activation = 'tanh', return_sequences = True))
  model.add(Dropout(rate = 0.2))
  model.add(TimeDistributed(Dense(units = 1)))

  model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'mean_absolute_error', metrics=['mae']) 

  es = EarlyStopping(monitor ='val_loss', mode ='min', verbose = 1, patience = 5)
  model_save_folder_path = '/content/drive/MyDrive/stock_2/lstm_private/lstm'
  model_path = model_save_folder_path + f'{code}.h5'
  mc = ModelCheckpoint(model_path, monitor ='val_loss', mode ='min', verbose = 100, save_best_only = True)

  model.fit(X_train, y_train, epochs = 500, batch_size = 2, verbose = 1, shuffle = False, callbacks = [es, mc], validation_data=(X_val, y_val))

  return model

In [14]:
for code in tqdm(stock_list['종목코드'].values):
  
  # 해당 종목의 데이터 가져온 후 주 단위 데이터로 나타내기
  start_date = '20210104'
  end_date = '20211105'
  stock_price = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
  stock_price = pd.merge(business_days, stock_price, how = 'outer')
  stock_price['weekday'] = stock_price.Date.apply(lambda x : x.weekday())
  stock_price['weeknum'] = stock_price.Date.apply(lambda x : x.strftime('%V'))
  stock_price.Close = stock_price.Close.ffill()
  stock_price = pd.pivot_table(data = stock_price, values = 'Close', columns = 'weekday', index = 'weeknum')

  X_train = stock_price.iloc[0:-12].to_numpy()
  X_val = stock_price.iloc[-12:-2].to_numpy()
  X_test = stock_price.iloc[-2].to_numpy()

  y_train = stock_price.iloc[1:-11].to_numpy()
  y_val = stock_price.iloc[-11:-1].to_numpy()
  y_test = stock_price.iloc[-1].to_numpy()

  # 스케일링 시행
  scaler = MinMaxScaler()

  X_train_scaled = scaler.fit_transform(X_train.reshape(-1, 1)).reshape(35, 5)
  X_val_scaled = scaler.fit_transform(X_val.reshape(-1, 1)).reshape(10, 5)
  X_test_scaled = scaler.fit_transform(X_test.reshape(-1, 1)).reshape(1, 5)

  y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(35, 5)
  y_val_scaled = scaler.fit_transform(y_val.reshape(-1, 1)).reshape(10, 5)
  y_test_scaled = scaler.fit_transform(y_test.reshape(-1, 1)).reshape(1, 5)

  X_train_scaled = np.reshape(X_train_scaled, (X_train_scaled.shape[0], X_train_scaled.shape[1]))
  X_val_scaled = np.reshape(X_val_scaled, (X_val_scaled.shape[0], X_val_scaled.shape[1]))
  X_test_scaled = np.reshape(X_test_scaled, (X_test_scaled.shape[0], X_test_scaled.shape[1]))
  
  # 모델 학습
  model = LSTM_model(X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled)
  model = keras.models.load_model(f"/content/drive/MyDrive/stock_2/lstm_private/lstm{code}.h5")

  # 예측
  y_pred = model.predict(X_test_scaled)
  y_pred = y_pred.reshape(1,5)
  y_pred = scaler.inverse_transform(y_pred)

  print(y_pred)
    
  for i in range (0, 5):
    submission.loc[i + 5, code] = y_pred[0][i]

 90%|█████████ | 334/370 [1:41:43<09:51, 16.42s/it]

[[11050.289 11050.33  11050.349 11050.356 11050.36 ]]
Epoch 1/500
17/18 [===========================>..] - ETA: 0s - loss: 0.2970 - mae: 0.2970
Epoch 00001: val_loss improved from inf to 0.20075, saving model to /content/drive/MyDrive/stock_2/lstm_private/lstm204270.h5
18/18 [==============================] - 7s 92ms/step - loss: 0.2906 - mae: 0.2906 - val_loss: 0.2008 - val_mae: 0.2008
Epoch 2/500
17/18 [===========================>..] - ETA: 0s - loss: 0.2012 - mae: 0.2012
Epoch 00002: val_loss improved from 0.20075 to 0.16693, saving model to /content/drive/MyDrive/stock_2/lstm_private/lstm204270.h5
18/18 [==============================] - 0s 19ms/step - loss: 0.1995 - mae: 0.1995 - val_loss: 0.1669 - val_mae: 0.1669
Epoch 3/500
18/18 [==============================] - ETA: 0s - loss: 0.1601 - mae: 0.1601
Epoch 00003: val_loss improved from 0.16693 to 0.16166, saving model to /content/drive/MyDrive/stock_2/lstm_private/lstm204270.h5
18/18 [==============================] - 0s 20ms/s

 91%|█████████ | 335/370 [1:42:02<10:07, 17.35s/it]

[[8230.429 8230.418 8230.392 8230.357 8230.32 ]]
Epoch 1/500
16/18 [=========================>....] - ETA: 0s - loss: 0.2270 - mae: 0.2270
Epoch 00001: val_loss improved from inf to 0.29597, saving model to /content/drive/MyDrive/stock_2/lstm_private/lstm032190.h5
18/18 [==============================] - 7s 113ms/step - loss: 0.2249 - mae: 0.2249 - val_loss: 0.2960 - val_mae: 0.2960
Epoch 2/500
17/18 [===========================>..] - ETA: 0s - loss: 0.2477 - mae: 0.2477
Epoch 00002: val_loss did not improve from 0.29597
18/18 [==============================] - 0s 16ms/step - loss: 0.2438 - mae: 0.2438 - val_loss: 0.4191 - val_mae: 0.4191
Epoch 3/500
17/18 [===========================>..] - ETA: 0s - loss: 0.1871 - mae: 0.1871
Epoch 00003: val_loss did not improve from 0.29597
18/18 [==============================] - 0s 16ms/step - loss: 0.1836 - mae: 0.1836 - val_loss: 0.3527 - val_mae: 0.3527
Epoch 4/500
17/18 [===========================>..] - ETA: 0s - loss: 0.2021 - mae: 0.2021
Ep

 91%|█████████ | 336/370 [1:42:15<09:05, 16.05s/it]

...

[[6780.211  6780.175  6780.166  6780.1694 6780.1743]]
Epoch 1/500
15/18 [========================>.....] - ETA: 0s - loss: 0.1860 - mae: 0.1860
Epoch 00001: val_loss improved from inf to 0.35441, saving model to /content/drive/MyDrive/stock_2/lstm_private/lstm110790.h5
18/18 [==============================] - 7s 112ms/step - loss: 0.1978 - mae: 0.1978 - val_loss: 0.3544 - val_mae: 0.3544
Epoch 2/500
18/18 [==============================] - ETA: 0s - loss: 0.3489 - mae: 0.3489
Epoch 00002: val_loss improved from 0.35441 to 0.20751, saving model to /content/drive/MyDrive/stock_2/lstm_private/lstm110790.h5
18/18 [==============================] - 0s 18ms/step - loss: 0.3489 - mae: 0.3489 - val_loss: 0.2075 - val_mae: 0.2075
Epoch 3/500
16/18 [=========================>....] - ETA: 0s - loss: 0.2408 - mae: 0.2408
Epoch 00003: val_loss did not improve from 0.20751
18/18 [==============================] - 0s 17ms/step - loss: 0.2430 - mae: 0.2430 - val_loss: 0.3039 - val_mae: 0.3039
Epoch 4/

100%|█████████▉| 369/370 [1:52:07<00:16, 16.60s/it]

[[41850.348 41850.43  41850.473 41850.496 41850.508]]
Epoch 1/500
17/18 [===========================>..] - ETA: 0s - loss: 0.2504 - mae: 0.2504
Epoch 00001: val_loss improved from inf to 0.32709, saving model to /content/drive/MyDrive/stock_2/lstm_private/lstm099320.h5
18/18 [==============================] - 7s 125ms/step - loss: 0.2479 - mae: 0.2479 - val_loss: 0.3271 - val_mae: 0.3271
Epoch 2/500
18/18 [==============================] - ETA: 0s - loss: 0.1699 - mae: 0.1699
Epoch 00002: val_loss did not improve from 0.32709
18/18 [==============================] - 0s 16ms/step - loss: 0.1699 - mae: 0.1699 - val_loss: 0.3308 - val_mae: 0.3308
Epoch 3/500
17/18 [===========================>..] - ETA: 0s - loss: 0.1429 - mae: 0.1429
Epoch 00003: val_loss improved from 0.32709 to 0.32296, saving model to /content/drive/MyDrive/stock_2/lstm_private/lstm099320.h5
18/18 [==============================] - 0s 19ms/step - loss: 0.1407 - mae: 0.1407 - val_loss: 0.3230 - val_mae: 0.3230
Epoch 4/

100%|██████████| 370/370 [1:52:29<00:00, 18.24s/it]

[[45550.207 45549.984 45549.85  45549.8   45549.77 ]]


In [15]:
submission.to_csv("/content/drive/MyDrive/stock_2/submission_lstm.csv", index = False)

In [16]:
submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,...,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,29031.150391,34949.527344,60538.609375,147345.093750,111453.304688,17469.105469,49575.292969,86004.140625,106451.554688,692694.3750,51529.449219,231932.187500,32569.326172,57721.960938,16484.052734,2326.078613,26282.048828,5189.926758,325292.78125,50389.949219,13563.660156,33216.542969,9202.543945,7992.673828,30689.556641,147056.671875,47460.871094,90294.015625,43123.515625,237088.203125,283312.81250,80414.671875,101079.914062,33657.183594,29248.957031,32904.140625,144094.531250,211437.968750,98650.250000,...,39437.800781,107312.460938,280971.75000,12886.985352,166732.953125,176933.062500,7000.216309,7423.602539,34653.246094,13423.220703,87672.015625,49801.320312,52837.816406,25240.775391,266838.906250,599604.1250,690400.8750,22415.619141,51939.621094,73142.601562,29789.208984,120044.750000,36751.691406,13384.135742,3330.374023,39076.417969,18893.841797,64474.242188,96248.914062,5686.166504,48887.996094,52655.742188,84022.804688,35135.164062,25325.923828,50950.511719,363006.12500,253253.453125,26440.554688,17727.314453
1,2021-11-02,29289.136719,34803.519531,60766.707031,148037.140625,117857.296875,17578.798828,50247.812500,86104.609375,106271.429688,694087.6875,51501.765625,232086.062500,32649.429688,58167.617188,16692.525391,2322.898682,26240.126953,5207.011719,324836.12500,50476.945312,13691.573242,33340.589844,9282.301758,7996.734863,30718.140625,147387.484375,47504.003906,88446.343750,43501.957031,237010.281250,283182.18750,80524.390625,101700.734375,33602.839844,29533.212891,33161.453125,143143.109375,212398.765625,98857.507812,...,39398.343750,107191.789062,281955.53125,12905.487305,166814.687500,177110.250000,7034.061035,7446.492188,34752.250000,13475.184570,88581.898438,49891.343750,53164.445312,25435.771484,268372.187500,595241.8125,680532.6250,22503.605469,52078.523438,78054.953125,29698.982422,119453.210938,38610.644531,13404.479492,3335.626465,39086.453125,18858.511719,66614.812500,97041.093750,5678.149414,48838.187500,53035.050781,84202.703125,35214.402344,25540.480469,51018.871094,363550.50000,253708.968750,26902.123047,17767.306641
2,2021-11-03,29514.246094,34479.003906,61024.742188,148051.390625,120138.453125,17507.365234,49964.750000,86128.765625,106084.632812,693952.2500,51469.277344,232813.359375,32668.880859,58467.578125,16761.492188,2326.717285,26234.367188,5215.158203,325097.68750,50361.484375,13664.996094,32981.839844,9358.370117,8000.112305,30551.076172,147437.093750,47538.949219,85873.070312,43343.019531,236078.031250,283340.78125,80418.296875,101038.335938,33525.082031,29553.812500,33507.398438,144001.625000,213299.828125,99180.492188,...,39354.996094,107167.656250,276152.06250,12901.310547,166916.015625,176985.812500,7092.813965,7444.621582,34848.285156,13480.919922,95292.929688,49967.949219,53358.531250,25479.222656,268077.937500,584903.1875,669984.1250,22543.580078,52166.343750,80796.156250,30687.285156,118804.335938,38161.800781,13533.425781,3347.665283,39077.832031,18844.121094,67112.835938,97533.367188,5685.486816,48717.132812,53510.109375,84069.000000,35448.722656,25626.072266,51633.035156,367167.93750,256133.656250,26677.708984,17794.000000
3,2021-11-04,29484.294922,34247.226562,61200.234375,147343.296875,120556.132812,17095.908203,49936.738281,86132.390625,106231.007812,692796.3125,51249.714844,230831.640625,32632.597656,58780.453125,16719.748047,2319.621582,25799.625000,5216.588867,324471.31250,50216.871094,13639.401367,33032.5